In [2]:
import sys
import os
import numpy as np

# path to folder to CityGeoTools
folder = "/home/gk/vscode/CGT_2/local_cgt"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/home/gk/vscode/CGT_2/.venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
import os
from rpyc_server.data_classes.InterfaceCityInformationModel import DataQueryInterface

os.environ['MONGO'] = '10.32.1.65:5100'
os.environ['POSTGRES'] = 'postgres:postgres@10.32.1.107/city_db_final'

model = DataQueryInterface('Tara', 32643, 26)

2023-12-15 17:35:51 INFO Tara - MobilityGraph matches specification.
2023-12-15 17:36:02 INFO Tara - Buildings matches specification.
2023-12-15 17:36:02 INFO Tara - Services matches specification.
2023-12-15 17:36:03 INFO Tara - PublicTransportStops matches specification.
2023-12-15 17:36:03 INFO Tara - ServiceTypes matches specification.
2023-12-15 17:36:04 INFO Tara - Blocks matches specification.
2023-12-15 17:36:04 INFO Tara - Municipalities matches specification.


In [2]:
local_crs= 32640
city_name='Пермь'

### Create CityInformationModel

To define own City Information Model, you need to create an instance of CityInformationModel class with specified attributes: 
   
**city_name** - string (name of city which you are going to work with)  
**city_crs** - integer (EPSG projection of city)   
**cwd** - relative path to CityGeoTools library 


In [3]:
city_model = BaseModel.CityInformationModel(city_name=city_name, city_crs=local_crs, cwd="../")
city_model

The object of class CityInformationModel contains attributes describing data required for successful method call.  
After creating own model these attributes equal to **None** by default.

In [4]:
city_model.get_all_attributes()

{'city_name': 'Пермь',
 'city_crs': 32640,
 'city_id': typing.Optional[int],
 'cwd': '../',
 'mode': 'user_mode',
 'MobilityGraph': None,
 'Buildings': None,
 'Services': None,
 'PublicTransportStops': None,
 'ServiceTypes': None,
 'RecreationalAreas': None,
 'Blocks': None,
 'Municipalities': None,
 'AdministrativeUnits': None,
 'ValueTypes': None,
 'SocialGroups': None,
 'SocialGroupsValueTypesLivingSituations': None,
 'LivingSituationsCityServiceTypes': None,
 'LivingSituations': None,
 'methods': <metrics.data.DataValidation.DataValidation at 0x7fd254379b50>}

### Update layers

To use CityGeoTools methods, it is necessary to load prepaid data to the model specifying attribute names and paths to files.  
Only **json**, **geojson** and **graphml** file formats are available for loading. Before data loading, all methods are marked as unavailable.  
Exception is weighted voronoi method, since it doesn't require any urban data.

In [5]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")


All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> unavailable
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> unavailable
blocks_accessibility --> unavailable


In [6]:
# pip install pyarrow

In [7]:

import geopandas as gpd

blocks = gpd.read_parquet(f'{city_name}_blocks.parquet').to_crs(4326)
blocks = blocks.drop(columns=['landuse'])
blocks['is_polygon'] = blocks['geometry'].apply(lambda x : type(x).__name__ == 'Polygon')
blocks = blocks.loc[blocks['is_polygon']].reset_index(drop=True)

In [8]:
buildings = gpd.read_parquet('migr_proj_buildings_cities.parquet').to_crs(4326)
mask = buildings['city'] == city_name
buildings = buildings.loc[mask]
buildings.rename(columns={'building_id': 'id'}, inplace=True)
buildings.to_file('buildings.geojson')

In [9]:
services = gpd.read_file('services_ALL.geojson').to_crs(4326)
mask = services['city'] == city_name
services = services.loc[mask]
# services.to_crs(local_crs, inplace=True)
services.rename(columns={'building_id': 'id', 'city_service_type_code': 'service_code'}, inplace=True)
services.to_file('services.geojson')

In [10]:
services

,id,city_service_type,service_code,city,city_id,capacity,is_capacity_real,geometry
0,1794409,Поликлиника,policlinics,Пермь,30,4851,False,POINT (56.17030 58.00241)
7,1780219,Больница,hospitals,Пермь,30,5650,False,POINT (56.25458 58.01804)
8,1782169,Больница,hospitals,Пермь,30,237,False,POINT (56.17894 57.97977)
9,1782625,Детский сад,kindergartens,Пермь,30,56,False,POINT (56.26343 57.97042)
10,1796534,Школа,schools,Пермь,30,960,False,POINT (56.31137 57.97535)
...,...,...,...,...,...,...,...,...
997,1824368,Детский сад,kindergartens,Пермь,30,85,False,POINT (56.32924 58.00420)
998,1795855,Детский сад,kindergartens,Пермь,30,187,False,POINT (56.40535 58.08584)
999,1781963,Детский сад,kindergartens,Пермь,30,218,False,POINT (56.19991 57.98610)
1000,1824044,Школа,schools,Пермь,30,699,False,POINT (56.24488 57.99903)


In [11]:
blocks = gpd.read_parquet(f'{city_name}_blocks.parquet')
blocks.to_file(f'{city_name}_blocks.geojson')

In [12]:
gdfb = gpd.read_file("services.geojson")

In [13]:
gdfb.head()

,id,city_service_type,service_code,city,city_id,capacity,is_capacity_real,geometry
0,1794409,Поликлиника,policlinics,Пермь,30,4851,False,POINT (56.17030 58.00241)
1,1780219,Больница,hospitals,Пермь,30,5650,False,POINT (56.25458 58.01804)
2,1782169,Больница,hospitals,Пермь,30,237,False,POINT (56.17894 57.97977)
3,1782625,Детский сад,kindergartens,Пермь,30,56,False,POINT (56.26343 57.97042)
4,1796534,Школа,schools,Пермь,30,960,False,POINT (56.31137 57.97535)


In [14]:
gdfb.iloc[:10].explore()

In [15]:
# GeoJSON
city_model.update_layer("Buildings", "buildings.geojson")
city_model.update_layer("Services", "services.geojson")
city_model.update_layer("Blocks", f"{city_name}_blocks.geojson")

#GraphML
city_model.update_layer("MobilityGraph", f"{city_name}.graphml")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Services layer...
Services layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!
Validation of MobilityGraph layer...
MobilityGraph layer loaded successfully!


In [16]:
#JSON
# city_model.update_layer("ServiceTypes", "service_types.json") 

In [17]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")

All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> available
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> unavailable
blocks_accessibility --> unavailable


In [18]:
import psycopg2 as pg
city_model.engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")

In [19]:
service_types_d = services.rename(columns={'city_service_type_code': 'code'})

In [20]:
import pandas as pd
q = "select id, code, name, public_transport_time_normative, walking_radius_normative "\
    "from city_service_types "\
    "where code in ('kindergartens', 'schools', 'policlinics', 'hospitals', 'multifunctional_centers') "\
    ""
gdf  = pd.read_sql(q, con=city_model.engine)
gdf

/tmp/ipykernel_1259173/4075018148.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gdf  = pd.read_sql(q, con=city_model.engine)


,id,code,name,public_transport_time_normative,walking_radius_normative
0,2,schools,Школа,NaN,500.0
1,5,policlinics,Поликлиника,NaN,1000.0
2,6,hospitals,Больница,60.0,NaN
3,1,kindergartens,Детский сад,NaN,300.0
4,70,multifunctional_centers,МФЦ,20.0,NaN


In [21]:
city_model.ServiceTypes = gdf

In [22]:
city_model.Services.service_code.unique()

array(['policlinics', 'hospitals', 'kindergartens', 'schools',
       'multifunctional_centers'], dtype=object)

In [23]:
# df = city_model.Services
# df2 = df[df[['id', 'service_code']].duplicated() == True]
# display(df2.sort_values(by='id').shape)

In [24]:
# city_model.Services[city_model.Services['service_code']=='kindergartens'].sort_values(by='id')

In [25]:
city_model.city_crs

32640

In [26]:
# city_model.Services.drop_duplicates(subset='id', inplace=True)
# city_model.Buildings.drop_duplicates(subset='id', inplace=True)

In [27]:
city_model.Services.drop_duplicates(subset=['id', 'service_code'], inplace=True)

In [28]:
from metrics.calculations.city_provision import CityProvision

Provisions_class = CityProvision(
    city_model,
    service_types=[
        "kindergartens",
        "schools",
        "hospitals",
        "policlinics",
        "multifunctional_centers",
    ],
    valuation_type="normative",
    year=2023,
    user_changes_services=None,
    user_selection_zone=None,
)

/home/gk/vscode/CGT_2/local_cgt/metrics/calculations/city_provision.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  self.demands = pd.read_sql(


In [29]:
r = Provisions_class.get_provisions()

kindergartens not loaded



Provisions['buildings'][f'kindergartens_service_demand_left_value_normative'].sum() 61697.0 
Provisions['services']['capacity_left'].sum(), 46448 
Provisions['normative_distance'] 300.0
schools not loaded

Provisions['buildings'][f'schools_service_demand_left_value_normative'].sum() 122728.0 
Provisions['services']['capacity_left'].sum(), 78239 
Provisions['normative_distance'] 500.0
hospitals not loaded

Provisions['buildings'][f'hospitals_service_demand_left_value_normative'].sum() 8742.0 
Provisions['services']['capacity_left'].sum(), 273142 
Provisions['normative_distance'] 60.0
policlinics not loaded

Provisions['buildings'][f'policlinics_service_demand_left_value_normative'].sum() 19830.0 
Provisions['services']['capacity_left'].sum(), 121425 
Provisions['normative_distance'] 1000.0
multifunctional_centers not loaded

Provisions['buildings'][f'multifunctional_centers_service_demand_left_value_normative'].sum() 12717.0 
Provisions['services']['capacity_left'].sum(), 16997 
Provis

In [30]:
list(city_model.Services.service_code.unique())

['policlinics',
 'hospitals',
 'kindergartens',
 'schools',
 'multifunctional_centers']

In [31]:
r.buildings

,geometry,id,functional_object_id,is_living,city,building_area,living_area,storeys_count,population_balanced,city_id,...,policlinics_supplyed_demands_within,policlinics_supplyed_demands_without,policlinics_provison_value,multifunctional_centers_service_demand_value_normative,multifunctional_centers_service_demand_left_value_normative,multifunctional_centers_supplyed_demands_within,multifunctional_centers_supplyed_demands_without,multifunctional_centers_provison_value,is_shown,total_provision_assessment
1774223,POINT (56.27488 58.02018),1774223,0,1,Пермь,437.074,1397.60,5.0,61,30,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.000000,True,0.400000
1774224,POINT (56.30124 58.02761),1774224,0,1,Пермь,990.324,5745.70,9.0,185,30,...,4.0,0.0,1.0,2.0,0.0,1.0,1.0,0.500000,True,0.645455
1774225,POINT (56.30131 58.02807),1774225,0,1,Пермь,574.620,4022.34,10.0,145,30,...,3.0,0.0,1.0,2.0,0.0,1.0,1.0,0.500000,True,0.500000
1774227,POINT (56.30044 58.02821),1774227,0,1,Пермь,1020.240,5729.40,9.0,220,30,...,4.0,0.0,1.0,3.0,0.0,1.0,2.0,0.333333,True,0.574359
1774228,POINT (56.30206 58.02700),1774228,0,1,Пермь,963.512,5800.69,9.0,221,30,...,4.0,0.0,1.0,3.0,0.0,0.0,3.0,0.000000,True,0.525926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836849,POINT (56.25936 58.01425),1836849,0,1,Пермь,1137.890,4779.12,6.0,184,30,...,0.0,4.0,0.0,2.0,0.0,0.0,2.0,0.000000,True,0.200000
1836850,POINT (56.26544 57.99086),1836850,0,1,Пермь,1665.910,29153.30,25.0,1186,30,...,24.0,0.0,1.0,15.0,0.0,0.0,15.0,0.000000,True,0.600000
1836851,POINT (56.26324 57.99083),1836851,0,1,Пермь,1675.500,29321.20,25.0,1146,30,...,23.0,0.0,1.0,15.0,0.0,0.0,15.0,0.000000,True,0.600000
1836852,POINT (56.32151 58.02967),1836852,0,1,Пермь,1221.330,8549.31,10.0,334,30,...,0.0,7.0,0.0,4.0,0.0,0.0,4.0,0.000000,True,0.200000


In [32]:
r.buildings.to_parquet(f'{city_name}_prov/{city_name}_prov_b.parquet')